In [ ]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt 
import numpy as np
from scipy import stats
from pandas.api.types import CategoricalDtype

import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 150)

# special matplotlib argument for improved plots
from matplotlib import rcParams
sns.set_style("whitegrid")
sns.set_context("poster")

In [ ]:
finaldf = pd.read_csv('/Users/Julia/Documents/bootcamp/DC_capstone/finaldf.csv', low_memory=False)
finaldf.head()

In [ ]:
# MODIFICATIONS

#new dataframe with timestamp instead of datetime for SALE_DATE
finaldf.SALE_DATE = pd.to_datetime(finaldf.SALE_DATE)

times = []
for i in finaldf['SALE_DATE']:
    times.append(datetime.timestamp(i))
finaldf['SALE_DATE'] = times
    
#dropping LATITUDE and LONGITUDE because these values zig-zag between properties.
#dropping CENSUS_TRACT because it's included in CENSUS_BLOCK.
dropdf = finaldf.drop(['LATITUDE', 'LONGITUDE', 'CENSUS_TRACT'], axis=1)

#remove spaces
strips=dropdf['CENSUS_BLOCK'].apply(str).str.replace(" ", "")
dropdf['CENSUS_BLOCK'] = strips

#change numerical to categorical
for col in ['SALE_NUM', 'SQUARE', 'CENSUS_BLOCK', 'USECODE', 'ZIPCODE']:
    dropdf[col] = dropdf[col].astype('category')


In [ ]:
# CHECKING DISTRIBUTIONS

# it does not appear that the variables would benefit from transformations.

def hist_all(df, target):
    #scaling numerical cols
    num_list = list(df.loc[:, (df.dtypes == np.float64)].columns)
    scaledf = pd.DataFrame()
    for col in num_list:
        scaledf[col] = (df[col] - np.mean(df[col])) / np.std(df[col])
    num_list.remove(target)
    
    #histograms for numerical facets
    fig, axs = plt.subplots(len(scaledf.columns)-3,1, figsize=(10,6*len(scaledf.columns)))
    plt.rc('xtick', labelsize='small') 
    plt.rc('ytick', labelsize='small') 
    for i, name in enumerate(num_list):
        if (str(name) != target):
            plt.subplot(len(num_list), 1, i+1)
            a = plt.hist(scaledf[name], alpha=.5, bins=20)
            a = plt.hist(scaledf[target], alpha=.5, bins=20)
            plt.xlabel(name)
            plt.ylabel(target)
            plt.title("Histogram of %s" % name)  
    plt.tight_layout()

hist_all(dropdf, 'PRICE')

In [ ]:
# CORRELATION MATRIX

num_df = dropdf.loc[:, (dropdf.dtypes == np.float64)]
num_df.drop('PRICE', axis=1)
num_df.corr()

In [ ]:
# HEATMAP

f, ax = plt.subplots(figsize=(10, 6))
corr = num_df.corr()
hm = sns.heatmap(round(corr,2), annot=True, ax=ax, cmap="coolwarm",fmt='.2f',
                 linewidths=.05)
f.subplots_adjust(top=0.93)
t= f.suptitle('Housing Facets Correlation Heatmap', fontsize=14)

In [ ]:
# scatterplots of numerical columns
num_list = list(dropdf.loc[:, (dropdf.dtypes == np.float64)].columns)
scaledf = pd.DataFrame()
for col in dropdf[num_list]:
    scaledf[col] = (dropdf[col] - np.mean(dropdf[col])) / np.std(dropdf[col])
        
sns.set(style="ticks")
sns.pairplot(scaledf)

#### Potential variables to drop
* kitchen and units .917
* bedrooms and room .845
* bathrooms and bedrooms .735
* bathrooms and rooms .729
* yr_origin and yr_ext .687
* bathrooms and GBA .577
* bedrooms and landarea .570
* rooms and landarea .561
* units and room .515
* kitchens and room .514

In [ ]:
# CENSUS_BLOCK and SQUARE are the most specific geographical facets. 

geo_facets = ['QUADRANT', 'WARD', 'NBHD', 'SUBNBHD', 'CENSUS_BLOCK', 'ZIPCODE', 'SQUARE']
for facet in geo_facets:
    print("The number of unique values in %s IS %f" % (facet, len(finaldf[facet].unique())))

## SCALING FOR REGRESSION:

In [ ]:
#scale numerical X variables
X_cols = dropdf.drop('PRICE', axis=1)

num_list = list(dropdf.loc[:, (dropdf.dtypes == np.float64)].columns)
num_list.remove('PRICE')

scaler = StandardScaler()
X_scaled = X_cols.copy()
X_scaled[num_list] = scaler.fit_transform(X_cols[num_list])

# create dummy variables
Xdum = pd.get_dummies(X_scaled, drop_first=True)

# dataframe for regression: regdf
regdf = Xdum.copy()
regdf['PRICE'] = dropdf['PRICE']

In [ ]:
print("The size of our frame without dummies is {}".format(finaldf.shape))
print("The size of our regression df is {}".format(regdf.shape))

## FULL MODEL REGRESSION AND OUTLIER REMOVAL WITH STATSMODELS:

In [ ]:
# regression function with sklearn using train/test:

X = regdf.drop('PRICE', axis=1).values
y = regdf['PRICE'].values

def linreg(X, y): 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)
    reg_all=LinearRegression()
    reg_all.fit(X_train, y_train)
    y_pred = reg_all.predict(X_test)
    print("Train R^2: {}".format(reg_all.score(X_train, y_train)))
    print("Test R^2: {}".format(reg_all.score(X_test, y_test)))
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test Root Mean Squared Error: {}".format(rmse))
    
linreg(X, y)

In [ ]:
# regression using statsmodels
import statsmodels.api as sm

y = regdf['PRICE']
X = regdf.drop('PRICE', axis=1)

X = sm.add_constant(X)
reg_all = sm.OLS(y, X).fit()
reg_all.summary()

In [ ]:
# leverage plot

import statsmodels.graphics.regressionplots as plots

fig = plots.influence_plot(reg_all, alpha=0.01, plot_alpha=0.2, fontsize="small")

In [ ]:
# residuals plot 

plt.hist(reg_all.get_influence().resid_studentized_external)

In [ ]:
#Remove outliers and high leverage points from dataframe:

# Pick leverage cutoff for outliers by approximation (.05)
high_leverage_points = np.where(reg_all.get_influence().hat_matrix_diag > 0.05)

# Based on the histogram of the studentized residuals, remove outside of normal dist 3 SD away from mean....
# or approximate (above 3 in this case)
high_studentized_resid = np.where(reg_all.get_influence().resid_studentized_external > 3)

remove = np.concatenate([high_leverage_points[0], high_studentized_resid[0]])

regdf_no_outiers = regdf.drop(remove)
print regdf.shape, regdf_no_outiers.shape

## FEATURE SELECTION:

#### Potential variables to drop (remove one from each set, compare, leave out. Then progress to the next pair): 
* ##### kitchens, units, bedrooms, room, bathrooms, bedrooms, yr_origin, yr_ext, GBA, landarea,  

* kitchen and units .917
* bedrooms and room .845
* bathrooms and bedrooms .735
* bathrooms and rooms .729
* yr_origin and yr_ext .687
* bathrooms and GBA .577
* bedrooms and landarea .570
* rooms and landarea .561
* units and room .515
* kitchens and room .514

In [ ]:
# linear regressions, removing variables in order until RMSE is optimized

X = regdf_no_outiers.drop('PRICE', axis=1).values
y = regdf_no_outiers['PRICE'].values

def linreg(X, y, vars_dropped): 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)
    reg_all=LinearRegression()
    reg_all.fit(X_train, y_train)
    y_pred = reg_all.predict(X_test)
    print("Train R^2: {}".format(reg_all.score(X_train, y_train)))
    print("Test R^2 for " vars_dropped " : {}".format(reg_all.score(X_test, y_test)))
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test Root Mean Squared Error for " vars_dropped ": {}".format(rmse))
    
linreg(X, y)

## OUTLIER REMOVAL REMOVAL FROM DF WITHOUT COLINEAR FEATURES:

In [ ]:
# regression function with sklearn using train/test:

# dataframe with best features
feat_df = regdf_no_outliers.drop(   , axis=1)
X = feat_df.drop('PRICE', axis=1).values
y = feat_df['PRICE'].values

def linreg(X, y): 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)
    reg_all=LinearRegression()
    reg_all.fit(X_train, y_train)
    y_pred = reg_all.predict(X_test)
    print("Test R^2: {}".format(reg_all.score(X_test, y_test)))
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test Root Mean Squared Error: {}".format(rmse))
    
linreg(X, y)

In [ ]:
# regression using statsmodels
import statsmodels.api as sm

y = feat_df['PRICE']
X = feat_df.drop('PRICE', axis=1)

X = sm.add_constant(X)
reg_all = sm.OLS(y, X).fit()
reg_all.summary()

In [ ]:
# leverage plot

import statsmodels.graphics.regressionplots as plots

fig = plots.influence_plot(reg_all, alpha=0.01, plot_alpha=0.2, fontsize="small")

In [ ]:
# residuals plot 

plt.hist(reg_all.get_influence().resid_studentized_external)

In [ ]:
#Remove outliers and high leverage points from dataframe:

# Pick leverage cutoff for outliers by approximation (.05)
high_leverage_points = np.where(reg_all.get_influence().hat_matrix_diag > 0.05)

# Based on the histogram of the studentized residuals, remove outside of normal dist 3 SD away from mean....
# or approximate (above 3 in this case)
high_studentized_resid = np.where(reg_all.get_influence().resid_studentized_external > 3)

remove = np.concatenate([high_leverage_points[0], high_studentized_resid[0]])

feat_no_outliers = feat_df.drop(remove)
print feat_df.shape, feat_no_outliers.shape

## REGULARIZATION WITH SKLEARN FOR FINAL MODEL:

In [ ]:
# ridge regression function
def ridgereg(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)
    space = np.linspace(0, 1, 10)
    param_grid = {'alpha': space}
    
    ridge = Ridge()
    gm_cv = GridSearchCV(ridge, param_grid, cv=5)
    gm_cv.fit(X_train, y_train)

    y_pred = gm_cv.predict(X_test)
    r2 = gm_cv.score(X_test, y_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Tuned Ridge alpha: {}".format(gm_cv.best_params_))
    print("Tuned Ridge R squared: {}".format(r2))
    print("Tuned Ridge MSE: {}".format(mse))

In [ ]:
# lasso regression function
def lassoreg(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=42)
    space = np.linspace(0, 1, 10)
    param_grid = {'alpha': space}
    
    lasso = Lasso()
    gm_cv = GridSearchCV(lasso, param_grid, cv=5)
    gm_cv.fit(X_train, y_train)

    y_pred = gm_cv.predict(X_test)
    r2 = gm_cv.score(X_test, y_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Tuned LASSO Alpha: {}".format(gm_cv.best_params_))
    print("Tuned LASSO R squared: {}".format(r2))
    print("Tuned LASSO MSE: {}".format(mse))

In [ ]:
#ElasticNet function
def elasticnet(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=42)
    space = np.linspace(0, 1, 10)
    param_grid = {'l1_ratio': space}
    
    elastic_net = ElasticNet()
    gm_cv = GridSearchCV(elastic_net, param_grid, cv=5)
    gm_cv.fit(X_train, y_train)

    y_pred = gm_cv.predict(X_test)
    r2 = gm_cv.score(X_test, y_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Tuned ElasticNet l1 ratio: {}".format(gm_cv.best_params_))
    print("Tuned ElasticNet R squared: {}".format(r2))
    print("Tuned ElasticNet MSE: {}".format(mse))

# WHAT SPECIFIC METRICS DESCRIBE OVER FITTING?
* compare training with test set. if model works significantly better on training set (R^2. MSE?), then it's overfitting. 

* "Regularization" process (correcting over fitting):
    * 1. check for outliers
    * 2. remove features
    * 3. try ridge lasso elasticnet
    * 4. go to other methods. 

In [ ]:
%%capture
#gather features
features = "+".join(df.columns - ["annual_inc"])

# get y and X dataframes based on this regression:
y, X = dmatrices('annual_inc ~' + features, df, return_type='dataframe')

# For each X, calculate VIF and save in dataframe
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

vif.round(1)